#### Download Normal HBB Yolo model and test on our dataset

In [2]:
import os
import shutil
from pathlib import Path

# CONFIG
SOURCE_DATASET = "./hbb_Birds_detect.v16i.yolov8" 
TARGET_DATASET = "./Birds-detect-HBB-Benchmark-Fixed"

def recreate_benchmark_dataset(source_path, target_path):
    print(f"Creating Benchmark Dataset at: {target_path}")
    
    # 1. Clean up old target if exists
    if os.path.exists(target_path):
        shutil.rmtree(target_path)
    os.makedirs(target_path)
    
    # 2. COPY IMAGES
    src_images = Path(source_path) / "test/images"
    dst_images = Path(target_path) / "test/images"
    shutil.copytree(src_images, dst_images)
    print(f"Copied images from {src_images}")

    # 3. CONVERT LABELS (Force Class 14)
    src_labels = Path(source_path) / "test/labels"
    dst_labels = Path(target_path) / "test/labels"
    os.makedirs(dst_labels)

    print(f"Processing labels from {src_labels}...")
    
    count = 0
    for lbl_file in src_labels.glob("*.txt"):
        with open(lbl_file, 'r') as f:
            lines = f.readlines()
            
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # Force first number to be 14
                # Keep coordinates (cx cy w h) same
                parts[0] = "14"
                new_lines.append(" ".join(parts) + "\n")
        
        if new_lines:
            with open(dst_labels / lbl_file.name, 'w') as f_out:
                f_out.writelines(new_lines)
            count += 1
            
    print(f"Converted {count} label files (Class 0 -> 14)")

    # 4. CREATE YAML
    yaml_content = f"""
path: {os.path.abspath(target_path)}
train: test/images
val: test/images
test: test/images
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird  # <--- 
"""
    yaml_path = Path(target_path) / "benchmark.yaml"
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
        
    print(f"Created YAML: {yaml_path}")
    return str(yaml_path)


yaml_file = recreate_benchmark_dataset("./hbb_Birds_detect.v16i.yolov8", "./Birds-detect-HBB-Benchmark-Fixed")

# VALIDATE IMMEDIATELY
from ultralytics import YOLO
base_model = YOLO("yolo11n.pt")

print("\nRunning Validation...")
metrics = base_model.val(
    data=yaml_file,
    split='test',
    device='cuda',
    workers=0,
    classes=[14],
    name="benchmark_coco_yolo"
)

print("\n" + "="*70)
print("BENCHMARK RESULTS")
print("="*70)
print(f"Standard YOLO11n mAP@50:    {metrics.box.map50:.4f}")
print(f"Standard YOLO11n Precision: {metrics.box.mp:.4f}")
print(f"Standard YOLO11n Recall:    {metrics.box.mr:.4f}")
print("="*70)


Creating Benchmark Dataset at: ./Birds-detect-HBB-Benchmark-Fixed
Copied images from hbb_Birds_detect.v16i.yolov8\test\images
Processing labels from hbb_Birds_detect.v16i.yolov8\test\labels...
Converted 270 label files (Class 0 -> 14)
Created YAML: Birds-detect-HBB-Benchmark-Fixed\benchmark.yaml

Running Validation...
Ultralytics 8.3.237  Python-3.13.5 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 2.91.7 MB/s, size: 45.8 KB)
val: Scanning C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\Birds-detect-HBB-Benchmark-Fixed\test\labels... 270 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 279/279 249.5it/s 1.1s0.1s
val: New cache created: C:\Users\Acer Nitro\Documents\CSC FILES\4th Year First Semester\Intellegent Systems\Deep Computer Vision\Bird detection\Birds-